In [1]:
#%pip install pandas pyarrow
import pandas as pd

# Load the Parquet file
articles = pd.read_parquet('articles.parquet', engine='pyarrow')
behaviors = pd.read_parquet('train/behaviors.parquet', engine='pyarrow')
history = pd.read_parquet('train/history.parquet', engine='pyarrow')

# bert = pd.read_parquet('bert_base_multilingual_cased.parquet', engine='pyarrow')
# display = bert['google-bert/bert-base-multilingual-cased']

In [2]:
articles[["ner_clusters", "entity_groups"]]

,ner_clusters,entity_groups
0,[],[]
1,"[Harry, James Hewitt]","[PER, PER]"
2,[],[]
3,[],[]
4,[],[]
...,...,...
11772,"[Hawaiis, Kilauea, Kilauea, Reuters, US Geolog...","[LOC, LOC, PER, ORG, ORG]"
11773,"[Alexandra Bøje, All England, Amalie Magelund,...","[PER, PROD, PER, PER, MISC, MISC, PER, PER, LO..."
11774,"[Cooper, Englemageren, Julie R, Julie R. Ølgaa...","[PER, PROD, PER, PER, PER, PER, MISC]"
11775,"[Dnepr, Kherson, Nova Kakhovka, Nova Kakhovka,...","[LOC, LOC, LOC, PROD, PER, LOC, ORG, ORG, LOC]"


In [ ]:
behaviors.sort_values('impression_time')

In [ ]:
history.head()

# Test wikipedia entity embeddings

In [2]:
with open("../../../../glove.840B.300d.txt", "r", encoding="utf8") as f:
    lines = f.readlines()
    print(lines[0]) 

, -0.082752 0.67204 -0.14987 -0.064983 0.056491 0.40228 0.0027747 -0.3311 -0.30691 2.0817 0.031819 0.013643 0.30265 0.0071297 -0.5819 -0.2774 -0.062254 1.1451 -0.24232 0.1235 -0.12243 0.33152 -0.006162 -0.30541 -0.13057 -0.054601 0.037083 -0.070552 0.5893 -0.30385 0.2898 -0.14653 -0.27052 0.37161 0.32031 -0.29125 0.0052483 -0.13212 -0.052736 0.087349 -0.26668 -0.16897 0.015162 -0.0083746 -0.14871 0.23413 -0.20719 -0.091386 0.40075 -0.17223 0.18145 0.37586 -0.28682 0.37289 -0.16185 0.18008 0.3032 -0.13216 0.18352 0.095759 0.094916 0.008289 0.11761 0.34046 0.03677 -0.29077 0.058303 -0.027814 0.082941 0.1862 -0.031494 0.27985 -0.074412 -0.13762 -0.21866 0.18138 0.040855 -0.113 0.24107 0.3657 -0.27525 -0.05684 0.34872 0.011884 0.14517 -0.71395 0.48497 0.14807 0.62287 0.20599 0.58379 -0.13438 0.40207 0.18311 0.28021 -0.42349 -0.25626 0.17715 -0.54095 0.16596 -0.036058 0.08499 -0.64989 0.075549 -0.28831 0.40626 -0.2802 0.094062 0.32406 0.28437 -0.26341 0.11553 0.071918 -0.47215 -0.18366 -0.3

In [3]:
entities = []
nr_articles_no_entities = 0
for ner_clusters in articles["ner_clusters"]:
    if len(ner_clusters) == 0:
        nr_articles_no_entities += 1
    for entity in ner_clusters:
        entities.append(entity)
unique_entities = set(entities)
print(f"Number of unique entities: {len(unique_entities)}")
print(f"Percentage of articles without entities: {(nr_articles_no_entities / len(articles)) * 100}%")

Number of unique entities: 33078
Percentage of articles without entities: 14.32453086524582%


In [4]:
found = 0
found_embeddings = []
for line in lines[1:]:
    entity = line.split()[0]
    if entity in unique_entities:
        found += 1
        found_embeddings.append(entity)
print(f"Percentage of entities found in the wikipedia embeddings: {(found/len(unique_entities)  * 100)}%")    

Percentage of entities found in the wikipedia embeddings: 20.39119656569321%


In [35]:
found

6745

In [7]:
# write to file
with open("entities2id.txt", "w", encoding="utf8") as f:
    for idx,entity in enumerate(found_embeddings):
        f.write(str(idx+1) +"\t" + entity + "\n")

In [28]:
len(unique_entities)

33078

In [22]:
1+1

2

## Create mergeimps.tsv (popularity)
The "mergeimps.tsv'' file needs to store the #click and #impression of each news with time information for building popularity signals. Specifically, we first need to divide the time into several buckets in units of one hour. Thus, since the MIND dataset contains user data in 6 weeks, the time periods can be divided into 2476 buckets. Then, we count the #impressions and #clicks of a news in each bucket and store the results in each line of ``mergeimps.tsv'', where the format is :

[News_ID] TAB [Bucket_ID] TAB #Cliks TAB #Impressions.
 
In MIND, since we did not obtain the click information for test data, which can substantially affect the estimation of popularity signals. Thus, I advise that you can use the validation dataset of MIND for evaluation.

In [ ]:
behaviors[:10]

In [ ]:
# anchor = trans2tsp('05/08/2023 11:59:59 PM')
# def parse_time_bucket(date):
#     tsp = trans2tsp(date)
#     tsp = tsp - anchor
#     tsp = tsp//(600 * 2.2)
#     return tsp


# Assuming 'impression_time' is already in datetime format, if not convert it
behaviors['impression_time'] = pd.to_datetime(behaviors['impression_time'])

# Set the start date of your data collection period
start_date = pd.to_datetime('2023-05-18')

# Calculate the hour buckets since the start of the period
behaviors['bucket_id'] = ((behaviors['impression_time'] - start_date).dt.total_seconds() // (600)).astype(int)

# Check if 'article_ids_inview' is already in the desired format
if not behaviors['article_ids_inview'].str.contains('N').any():
    # For each line in the 'article_ids_inview' column, add 'N' upfront
    # Add 'N' before each 'article_ids_inview', replace each space with ' N' and remove square brackets
    behaviors['article_ids_inview'] = behaviors['article_ids_inview'].apply(lambda x: 'N' + str(x))
    behaviors['article_ids_inview'] = behaviors['article_ids_inview'].str.replace(' ', ' N').str.replace('[', '').str.replace(']', '').str.replace('\n', '')

# Same for article_ids_clicked
if not behaviors['article_ids_clicked'].str.contains('N').any():
    behaviors['article_ids_clicked'] = behaviors['article_ids_clicked'].apply(lambda x: 'N' + str(x) )
    behaviors['article_ids_clicked'] = behaviors['article_ids_clicked'].str.replace(' ', ' N').str.replace('[', '').str.replace(']', '')

behaviors[["bucket_id", "impression_time", "article_ids_inview", "article_ids_clicked"]].head()

In [ ]:
behaviors[["bucket_id", "impression_time", "article_ids_inview", "article_ids_clicked"]].sort_values('bucket_id')

In [ ]:
double_dict = {}

for i, row in behaviors.iterrows():

    for article_id in row["article_ids_inview"].split(' '):
        if str(article_id) + "-" + str(row["bucket_id"]) not in double_dict:
            double_dict[str(article_id) + "-" + str(row["bucket_id"])] = {"article_id": article_id, "bucket_id": row["bucket_id"], "clicks": 0, "impressions": 0}
        double_dict[str(article_id) + "-" + str(row["bucket_id"])]["impressions"] += 1
            
    for article_id in row["article_ids_clicked"].split(' '):
        if str(article_id) + "-" + str(row["bucket_id"]) not in double_dict:
            double_dict[str(article_id) + "-" + str(row["bucket_id"])] = {"article_id": article_id, "bucket_id": row["bucket_id"], "clicks": 0, "impressions": 0}
        double_dict[str(article_id) + "-" + str(row["bucket_id"])]["clicks"] += 1
        
mergeimps = pd.DataFrame(double_dict.values())

In [ ]:
# mergeimps.to_excel("mergeimps.xlsx", index=False)
mergeimps.to_csv("../popularity/mergeimps.tsv", sep="\t", index=False, header=False)

In [ ]:
# tests
test = mergeimps.groupby("article_id").agg({"clicks": "sum", "impressions": "sum", "bucket_id" :["nunique", "count"]}).reset_index()
test

## Create V21UrlDocs22_title_json.tsv (necessary?)
For implementing our method based on MIND, the "V21UrlDocs22_title_json.tsv'' needs to store all news in the training/val/test set. Each line stores the information of a single news article, where format is based on 

[News_ID] TAB [Category] TAB [Subcategory] TAB [Title] TAB [Url] TAB [Abstract] TAB [Body].
 
You can modify my codes to avoid the load of some unnecessary news data (e.g., Url, Abstract, and Body)

## Create train/val/test.tsv

In [ ]:
def process_ids(row):
    clicked_ids = set(row['article_ids_clicked'].split())
    inview_ids = row['article_ids_inview'].split()
    processed_ids = [id for id in inview_ids if id not in clicked_ids]
    return ' '.join(processed_ids)

# Convert 'impression_time' to string
behaviors['impression_time'] = behaviors['impression_time'].astype(str)

# Check if 'impression_time' is already in the desired format
if not behaviors['impression_time'].str.contains('/').any():
    # Convert 'impression_time' to datetime
    behaviors['impression_time'] = pd.to_datetime(behaviors['impression_time'])

    # Format 'impression_time' in the desired format
    behaviors['impression_time'] = behaviors['impression_time'].dt.strftime('%m/%d/%Y %I:%M:%S %p')

# Add 'U' before every 'user_id' if it doesn't already start with 'U'
behaviors['user_id'] = behaviors['user_id'].apply(lambda x: 'U' + str(x) if not str(x).startswith('U') else str(x))

# Convert 'article_ids_inview' to string
behaviors['article_ids_inview'] = behaviors['article_ids_inview'].astype(str)

# Check if 'article_ids_inview' is already in the desired format
if not behaviors['article_ids_inview'].str.contains('N').any():
    # For each line in the 'article_ids_inview' column, add 'N' upfront
    # Add 'N' before each 'article_ids_inview', replace each space with ' N' and remove square brackets
    behaviors['article_ids_inview'] = behaviors['article_ids_inview'].apply(lambda x: 'N' + str(x))
    behaviors['article_ids_inview'] = behaviors['article_ids_inview'].str.replace(' ', ' N').str.replace('[', '').str.replace(']', '').str.replace('\n', '')

# Same for article_ids_clicked
if not behaviors['article_ids_clicked'].str.contains('N').any():
    behaviors['article_ids_clicked'] = behaviors['article_ids_clicked'].apply(lambda x: 'N' + str(x) )
    behaviors['article_ids_clicked'] = behaviors['article_ids_clicked'].str.replace(' ', ' N').str.replace('[', '').str.replace(']', '')

# Select the required columns
train_file = behaviors[['user_id', 'impression_time', 'article_ids_clicked', 'article_ids_inview']]

# Write to a new .tsv file without index
train_file.to_csv('../train.tsv', sep='\t', index=False, header=True)

# Read the file and print the head
df = pd.read_csv('../train.tsv', sep='\t')
df['article_ids_inview'] = df['article_ids_inview'].str.replace('"', '')
df['article_ids_inview'] = df.apply(process_ids, axis=1)

df['tot_clicks'] = df.apply(lambda row: (row['article_ids_clicked'], row['impression_time']),axis=1)
df['positive'] = df['article_ids_clicked']
df['negatives'] = df['article_ids_inview'].to_list()
df['bucket'] = df['impression_time']

df_grouped = df.groupby('user_id').agg({'tot_clicks': lambda x: x.tolist()}).reset_index()

df = df.drop('tot_clicks', axis=1).merge(df_grouped[['user_id', 'tot_clicks']], on='user_id', how='left')

docs = df[['user_id', 'tot_clicks', 'positive', 'negatives', 'bucket']]


# Write the DataFrame back to the .tsv file
train_split = docs[:int(len(docs)*0.8)]
val_split = docs[int(len(docs)*0.8) : int(len(docs)*0.9)]
test_split = docs[int(len(docs)*0.9) : ]

train_split.to_csv('../train.tsv', sep='\t', index=False, header=False)
val_split.to_csv('../val.tsv', sep='\t', index=False, header=False)
test_split.to_csv('../test.tsv', sep='\t', index=False, header=False)



In [ ]:
docs

## Create docs.tsv

In [ ]:

# Select the required columns
articles['body'] = articles['body'].str.replace('\n', ' ')
articles['title'] = articles['title'].str.replace('\n', ' ')
articles['body'] = articles['body'].str.replace('\t', ' ')
# articles['body'] = articles['body'].fillna('')
# articles['category_str'] = articles['category_str'].fillna('')
articles['topics'] = articles['topics'].astype(str).str.replace('\n', ' ')
articles['article_id'] = articles['article_id'].apply(lambda x: 'N' + str(x) if not str(x).startswith('N') else str(x))
docs_file = articles[['article_id', 'title', 'category_str', 'subcategory', 'body', 'topics']]

# Write to a new .tsv file without index
docs_file.to_csv('../docs.tsv', sep='\t', index=False, header=False)

## Create docs_pub_time.tsv

In [ ]:
articles['article_id'] = articles['article_id'].apply(lambda x: 'N' + str(x) if not str(x).startswith('N') else str(x))

# Convert 'impression_time' to string
articles['published_time'] = articles['published_time'].astype(str)

# Check if 'impression_time' is already in the desired format
if not articles['published_time'].str.contains('/').any():
    # Convert 'impression_time' to datetime
    articles['published_time'] = pd.to_datetime(articles['published_time'])

    # Format 'impression_time' in the desired format
    articles['published_time'] = articles['published_time'].dt.strftime('%m/%d/%Y %I:%M:%S %p')
pubdocs_file = articles[['article_id', 'published_time']]

# Write to a new .tsv file without index
pubdocs_file.to_csv('../popularity/docs_pub_time.tsv', sep='\t', index=False, header=False)

## Create entities